# Build adjacency matrix for comparison with other methods
**Authorship:** Adam Klie (last updated: 08/07/2023)<br>
***
**Description:** This notebook 
***

In [36]:
# Conversion libraries and Seurat
library(SeuratDisk)
library(SeuratData)
library(Seurat)
library(Signac)

# plotting and data science packages
library(tidyverse)
library(cowplot)
library(patchwork)

# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

In [37]:
# TODO: change to relative path based on your dataset
seurat_dir <- "/cellar/users/aklie/data/igvf/topic_grn_links/seurat"
dataset_name <- "igvf_b01_LeftCortex"
results_dir <- "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/hdwgcna/results"
analysis <- "network_construction"

In [38]:
# Parameters
CELL.TYPES <- "all"
ASSAY <- "RNA"
NORMALIZATION <- "ND"
NN <- 25
GENES <- "all"
PROCESS_W_SEURAT <- TRUE

# Output fie path
NAME <- paste0(CELL.TYPES, "cells_", GENES, "genes_", NORMALIZATION, "_", NN, "neighbors")
OUT <- file.path(results_dir, dataset_name, "Microglia", "CASTJ_filtered", analysis)
OUT

[1] "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/hdwgcna/results/igvf_b01_LeftCortex/Microglia/CASTJ_filtered/network_construction"

In [39]:
# Load in the pre-processed data
seurat_obj <- readRDS(file=file.path(OUT, paste0(NAME, "_hdWGCNA.rds")))

In [40]:
# Grab power
power <- 14

In [41]:
dat <- get(NAME, seurat_obj@misc)$datExpr

In [42]:
adj <- WGCNA::adjacency(
    datExpr=dat,
    type="signed",
    power=power
)

In [43]:
getLinkList <- function(weightMatrix, reportMax=NULL, threshold=0) {
    if(!is.numeric(threshold)) {
    	stop("threshold must be a number.")
    }

	# Only process weights off-diagonal
  regulatorsInTargets <- rownames(weightMatrix)[rownames(weightMatrix) %in% colnames(weightMatrix)]
  if(length(regulatorsInTargets)==1) weightMatrix[regulatorsInTargets, regulatorsInTargets] <- NA
  if(length(regulatorsInTargets)>1) diag(weightMatrix[regulatorsInTargets, regulatorsInTargets]) <- NA

  linkList <- reshape2::melt(weightMatrix, na.rm=TRUE)
  colnames(linkList) <- c("regulatoryGene", "targetGene", "weight")
  linkList <- linkList[linkList$weight>=threshold,]
  linkList <- linkList[order(linkList$weight, decreasing=TRUE),]
  
  if(!is.null(reportMax)) {
  	linkList <- linkList[1:min(nrow(linkList), reportMax),]
  }
  rownames(linkList) <- NULL
  
  uniquePairs <- nrow(unique(linkList[,c("regulatoryGene", "targetGene")]))
  if(uniquePairs < nrow(linkList)) 
    warning("There might be duplicated regulator-target (gene id/name) pairs.")
  
  return(linkList)
}

In [44]:
link.list <- getLinkList(adj)

In [ ]:
# Save the adjacency list of the left cortex to a file
write.table(link.list, file = file.path(OUT, "adj.tsv"), sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

# DONE!

---